In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/07 23:46:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
from hyperopt import fmin, tpe, hp, space_eval, Trials, STATUS_OK

def objective(args):
    case, val = args
    if case == 'case 1':
        return val
    else:
        return val ** 2
space = hp.choice('a',
    [
        ('case 1', 1 + hp.lognormal('c1', 0, 1)),
        ('case 2', hp.uniform('c2', -10, 10))
    ]
)
best = fmin(objective, space, algo=tpe.suggest, max_evals=100)

print('best:', best)
print(space_eval(space, best))

100%|██████████| 100/100 [00:00<00:00, 197.24trial/s, best loss: 0.00017884417953651667]
best: {'a': 1, 'c2': -0.013373263608278896}
('case 2', -0.013373263608278896)


In [11]:
def objective(x):
    return { 'loss': x ** 2, 'status': STATUS_OK }
trials = Trials()
space = hp.uniform('x', -10, 10)
best = fmin(objective, space, algo=tpe.suggest, max_evals=100, trials=trials)
print('best:', best)
trials.trials

100%|██████████| 100/100 [00:00<00:00, 359.01trial/s, best loss: 0.003928639209310129]
best: {'x': 0.06267885775371253}


[{'state': 2,
  'tid': 0,
  'spec': None,
  'result': {'loss': 33.86630664965523, 'status': 'ok'},
  'misc': {'tid': 0,
   'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'workdir': None,
   'idxs': {'x': [0]},
   'vals': {'x': [5.819476492748745]}},
  'exp_key': None,
  'owner': None,
  'version': 0,
  'book_time': datetime.datetime(2023, 6, 8, 0, 18, 19, 141000),
  'refresh_time': datetime.datetime(2023, 6, 8, 0, 18, 19, 141000)},
 {'state': 2,
  'tid': 1,
  'spec': None,
  'result': {'loss': 0.952809430980199, 'status': 'ok'},
  'misc': {'tid': 1,
   'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'workdir': None,
   'idxs': {'x': [1]},
   'vals': {'x': [-0.9761195782178529]}},
  'exp_key': None,
  'owner': None,
  'version': 0,
  'book_time': datetime.datetime(2023, 6, 8, 0, 18, 19, 142000),
  'refresh_time': datetime.datetime(2023, 6, 8, 0, 18, 19, 142000)},
 {'state': 2,
  'tid': 2,
  'spec': None,
  'result': {'loss': 61.41053187851465, 'status': 'ok'},
  'misc': {'tid':

In [12]:
len(trials.results), trials.results

(100,
 [{'loss': 33.86630664965523, 'status': 'ok'},
  {'loss': 0.952809430980199, 'status': 'ok'},
  {'loss': 61.41053187851465, 'status': 'ok'},
  {'loss': 22.36710408916147, 'status': 'ok'},
  {'loss': 54.48979044693625, 'status': 'ok'},
  {'loss': 68.7235729041731, 'status': 'ok'},
  {'loss': 6.3919929206901775, 'status': 'ok'},
  {'loss': 10.424757502774469, 'status': 'ok'},
  {'loss': 81.32763905830703, 'status': 'ok'},
  {'loss': 82.15758350876887, 'status': 'ok'},
  {'loss': 5.517612284654886, 'status': 'ok'},
  {'loss': 2.2764281731956717, 'status': 'ok'},
  {'loss': 2.783117273696866, 'status': 'ok'},
  {'loss': 50.38279812291747, 'status': 'ok'},
  {'loss': 15.774446807333081, 'status': 'ok'},
  {'loss': 5.997045505500045, 'status': 'ok'},
  {'loss': 35.21181990652498, 'status': 'ok'},
  {'loss': 32.27496269417441, 'status': 'ok'},
  {'loss': 65.50870932950966, 'status': 'ok'},
  {'loss': 97.69613492875358, 'status': 'ok'},
  {'loss': 0.08963755182359379, 'status': 'ok'},
  

In [14]:
import numpy as np

min(trials.losses()), np.argmin(trials.losses()), trials.losses()

(0.003928639209310129,
 79,
 [33.86630664965523,
  0.952809430980199,
  61.41053187851465,
  22.36710408916147,
  54.48979044693625,
  68.7235729041731,
  6.3919929206901775,
  10.424757502774469,
  81.32763905830703,
  82.15758350876887,
  5.517612284654886,
  2.2764281731956717,
  2.783117273696866,
  50.38279812291747,
  15.774446807333081,
  5.997045505500045,
  35.21181990652498,
  32.27496269417441,
  65.50870932950966,
  97.69613492875358,
  0.08963755182359379,
  0.03281780484285516,
  0.8648256147425514,
  28.229339726479783,
  17.149746875172006,
  0.005864888966625484,
  0.0167107535149484,
  0.8615594995871505,
  38.19377067672657,
  20.914113836464995,
  12.652228856714327,
  93.9525804976384,
  63.19711490562999,
  1.6787146345098904,
  41.482918676675354,
  6.98210361813542,
  8.797196473169555,
  0.7124367764929987,
  12.49282591763556,
  2.682543598989048,
  4.4657953789415235,
  21.970699110940444,
  26.359852721247176,
  49.92229522046203,
  5.825661662706104,
  0.22

In [17]:
def objective_func(search_dict):
    x = search_dict['max_depth']
    y = search_dict['learning_rate']
    return { 'loss': x ** 2 + y * 20, 'status': STATUS_OK, 'x': x, 'y': y}
search_space = { 
                    'max_depth': hp.quniform('max_depth',  5, 15, 1),
                    'learning_rate': hp.uniform('learning_rate', 0.01, 0.1)
                }
algo = tpe.suggest
trials = Trials()
best = fmin(fn=objective_func, space=search_space, algo=algo, max_evals=5, trials=trials)
print('best:', best)

100%|██████████| 5/5 [00:00<00:00, 627.25trial/s, best loss: 26.24757042382431]
best: {'learning_rate': 0.062378521191215515, 'max_depth': 5.0}


In [18]:
trials.results

[{'loss': 101.32642809432636,
  'status': 'ok',
  'x': 10.0,
  'y': 0.0663214047163178},
 {'loss': 65.5999387146611,
  'status': 'ok',
  'x': 8.0,
  'y': 0.07999693573305537},
 {'loss': 26.24757042382431,
  'status': 'ok',
  'x': 5.0,
  'y': 0.062378521191215515},
 {'loss': 36.701171850839266,
  'status': 'ok',
  'x': 6.0,
  'y': 0.03505859254196322},
 {'loss': 122.98276107166939,
  'status': 'ok',
  'x': 11.0,
  'y': 0.09913805358346947}]